In [1]:
from fins3666.defines import *
from fins3666.utils import format_market_data, current_fx_data
from fins3666.portfolio import Portfolio
from fins3666.models.carry_trade import generate_orders

startTime = datetime(2001, 1, 1)

dates = mkt.loc[mkt['Timestamp'] >= startTime, 'Timestamp']
endTime = dates.iloc[-1]


fx_start = pd.DataFrame(current_fx_data(fx, startTime))


startingAssets = [
    {'asset': 'USD', 'units': 0, 'unit_value_USD': 1.0}]

date = startTime
currentData = current_fx_data(fx, date)

SamplePortfolio = Portfolio(startTime, startingAssets, currentData)


print(SamplePortfolio)
account_reference = ACCOUNT_SIZE_USD
prev_month = startTime.month

for date in dates:
    month = date.month
    yields = mkt[mkt['Timestamp'] <= date]

    fx_rates = current_fx_data(fx, date)

    SamplePortfolio.updateMarketData(
        currentData=fx_rates, yields=yields)

    # Loop that runs once per month
    if (month != prev_month):
        prev_month = month
        # determine which assets to long buy and which to short
        current_positions = SamplePortfolio.summary()
        fx_arrays = SamplePortfolio.forex_spreads

        newOrders = generate_orders(
            yields=yields,
            current_positions=SamplePortfolio.summary(),
            fx=fx_arrays,
            size=account_reference)

        for order in newOrders:
            SamplePortfolio.new_order(order)

        print(SamplePortfolio)

        account_reference = index_inflation(ACCOUNT_SIZE_USD, date)
        # account_reference = SamplePortfolio.longValue
    SamplePortfolio.update(date)


print(SamplePortfolio)



                                         Portfolio Overview                                         
Asset    Value      Units          Total Value (USD)           Position       Yield p.a      Weight
USD      $1.00             0     $              0.00             Long           0.00%         nan%

Portfolio - 1 assets
Timestamp: 2001-01-01 00:00:00,
Net Value ($USD): $0.00


                                         Portfolio Overview                                         
Asset    Value      Units          Total Value (USD)           Position       Yield p.a      Weight
USD      $1.00             0     $              0.00             Long           0.00%         nan%

Portfolio - 1 assets
Timestamp: 2001-01-31 00:00:00,
Net Value ($USD): $0.00


                                         Portfolio Overview                                         
Asset    Value      Units          Total Value (USD)           Position       Yield p.a      Weight
USD      $1.00             0     $   

KeyboardInterrupt: 

In [ ]:

MARGIN = 1.2e6

balance = SamplePortfolio.balance + MARGIN

benchmark = benchmarks.loc[benchmarks['Timestamp'].isin(
    balance.index), ['SP500', 'Timestamp']]

benchmark = pd.Series(benchmark['SP500'].tolist(),
                      index=benchmark['Timestamp'].tolist())/100+1

benchmark = benchmark*MARGIN

In [ ]:


# Ensure the comparison is done for each timestamp in benchmarks
rf_rate = mkt.loc[mkt['Timestamp'].isin(
    balance.index), ['USD LONG', 'Timestamp']]

rf_rate_pa = pd.Series(
    (rf_rate['USD LONG']/100).tolist(), index=rf_rate['Timestamp'])

test = Portfolio.metrics(balance, benchmark, rf_rate_pa)

In [ ]:
# print(f'{test['df']}')
print(f'Sharpe Ratio: {test['Sharpe Ratio']}')
print(f'Sortino Ratio: {test['Sortino Ratio']}')
print(f'Volatility: {test['Volatility']}')
print(f'Alpha: {test['Alpha']}')
print(f'Beta: {test['Beta']}')
print(f'Max Drawdown: {test['Max Drawdown']}')
print(f'Beta: {test['Beta']}')


test['df'].to_csv('results.csv')
test['df']['portfolio_return'].mean()*12

Sharpe Ratio: 1.4483178587277579
Sortino Ratio: 1.8200004142738446
Volatility: 0.26506470720465986
Alpha: 7.766482486648258
Beta: 0.6850498978192419
Max Drawdown: -0.7215850079368967
Beta: 0.6850498978192419


np.float64(0.14243138454060644)

In [ ]:
print(SamplePortfolio.trades)
SamplePortfolio.trades.to_csv('data/processed/trades.csv')

      Timestamp Asset Buy/Sell         Units  UnitPrice  Balance BaseCurrency  \
0    2001-02-01   JPY      NaN  4.612000e+08   0.075716      NaN          NOK   
1    2001-02-01   CHF      NaN  7.126903e+03   5.356022      NaN          NOK   
2    2001-02-01   CHF      NaN  6.512073e+06   1.366910      NaN          NZD   
3    2001-02-01   SEK      NaN  1.069937e+05   0.235143      NaN          NZD   
4    2001-02-01   SEK      NaN  3.777301e+07   0.157426      NaN          CAD   
...         ...   ...      ...           ...        ...      ...          ...   
1823 2025-03-03   JPY      NaN  1.932015e+06   0.006216      NaN          USD   
1824 2025-03-03   CHF      NaN  1.285073e+04   1.112595      NaN          USD   
1825 2025-03-03   NZD      NaN  7.935562e+03   0.609100      NaN          USD   
1826 2025-03-03   SEK      NaN  4.485091e+04   0.094403      NaN          USD   
1827 2025-03-03   GBP      NaN  3.511990e+03   1.263900      NaN          USD   

        OrderValue  Forecas

In [ ]:
print(SamplePortfolio.trade_log)
print(SamplePortfolio.ledger.head())
print(SamplePortfolio.trades)

['2001-02-01T00:00:00: \t Order <62c189b2-28d0-11f0-9995-00155dd4bcc2> [SELL 461200000.0 JPY @ MARKET (ID: 62c189b2)] EXECUTED @ 0.07572NOK', '2001-02-01T00:00:00: \t Order <62c18f98-28d0-11f0-9995-00155dd4bcc2> [SELL 7126.9027 CHF @ MARKET (ID: 62c18f98)] EXECUTED @ 5.35602NOK', '2001-02-01T00:00:00: \t Order <62c19542-28d0-11f0-9995-00155dd4bcc2> [SELL 6512073.0973 CHF @ MARKET (ID: 62c19542)] EXECUTED @ 1.36691NZD', '2001-02-01T00:00:00: \t Order <62c197cc-28d0-11f0-9995-00155dd4bcc2> [SELL 106993.7472 SEK @ MARKET (ID: 62c197cc)] EXECUTED @ 0.23514NZD', '2001-02-01T00:00:00: \t Order <62c19bfa-28d0-11f0-9995-00155dd4bcc2> [SELL 37773006.2528 SEK @ MARKET (ID: 62c19bfa)] EXECUTED @ 0.15743CAD', '2001-03-01T00:00:00: \t Order <62df3a5c-28d0-11f0-9995-00155dd4bcc2> [SELL 14056373.8798 JPY @ MARKET (ID: 62df3a5c)] EXECUTED @ 0.07549NOK', '2001-03-01T00:00:00: \t Order <62df4010-28d0-11f0-9995-00155dd4bcc2> [SELL 2909517.8498 JPY @ MARKET (ID: 62df4010)] EXECUTED @ 0.01968NZD', '2001-03